<a href="https://colab.research.google.com/github/XenosDar/EventPal/blob/milestone1/eventappmilestone1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import bcrypt

def connect_db():
    return sqlite3.connect("event_planner.db")

def create_tables():
    conn = connect_db()
    cursor = conn.cursor()

    # Users table for authentication
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            user_id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE NOT NULL,
            password TEXT NOT NULL
        )
    ''')

    # Events table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS events (
            event_id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER,
            event_name TEXT NOT NULL,
            event_date TEXT NOT NULL,
            event_description TEXT,
            FOREIGN KEY (user_id) REFERENCES users(user_id)
        )
    ''')

    conn.commit()
    conn.close()

def hash_password(password):
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt()).decode('utf-8')

def check_password(stored_password, provided_password):
    return bcrypt.checkpw(provided_password.encode('utf-8'), stored_password.encode('utf-8'))

def register_user(username, password):
    conn = connect_db()
    cursor = conn.cursor()
    hashed_password = hash_password(password)

    try:
        cursor.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, hashed_password))
        conn.commit()
        print("User registered successfully!")
    except sqlite3.IntegrityError:
        print("Error: Username already exists.")

    conn.close()

def login_user(username, password):
    conn = connect_db()
    cursor = conn.cursor()

    cursor.execute("SELECT user_id, password FROM users WHERE username = ?", (username,))
    user = cursor.fetchone()

    conn.close()

    if user and check_password(user[1], password):
        print("Login successful!")
        return user[0]  # Return user_id
    else:
        print("Invalid credentials.")
        return None

def create_event(user_id, event_name, event_date, event_description):
    conn = connect_db()
    cursor = conn.cursor()

    cursor.execute("INSERT INTO events (user_id, event_name, event_date, event_description) VALUES (?, ?, ?, ?)",
                   (user_id, event_name, event_date, event_description))
    conn.commit()
    conn.close()
    print("Event created successfully!")

def edit_event(event_id, event_name=None, event_date=None, event_description=None):
    if not any([event_name, event_date, event_description]):
        print("No updates provided.")
        return

    conn = connect_db()
    cursor = conn.cursor()

    update_fields = []
    params = []

    if event_name:
        update_fields.append("event_name = ?")
        params.append(event_name)
    if event_date:
        update_fields.append("event_date = ?")
        params.append(event_date)
    if event_description:
        update_fields.append("event_description = ?")
        params.append(event_description)

    params.append(event_id)

    query = f"UPDATE events SET {', '.join(update_fields)} WHERE event_id = ?"
    cursor.execute(query, params)
    conn.commit()
    conn.close()
    print("Event updated successfully!")

# Initialize the database
def initialize_app():
    create_tables()
    print("Database initialized!")

# Run this once to set up tables
initialize_app()

Database initialized!


In [ ]:
register_user("test_user", "password123")

User registered successfully!


In [ ]:
!pip install bcrypt



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 3.8 MB/s eta 0:00:00


In [ ]:
user_id = login_user("test_user", "password123")
print(f"Logged in user ID: {user_id}")

Login successful!
Logged in user ID: 1


In [ ]:
if user_id:
    create_event(user_id, "Team Meeting", "2025-03-10", "Discuss project updates")

Event created successfully!


In [ ]:
conn = connect_db()
cursor = conn.cursor()
cursor.execute("SELECT * FROM events")
print(cursor.fetchall())  # Should show event details
conn.close()

[(1, 1, 'Team Meeting', '2025-03-10', 'Discuss project updates')]
